# Subscribe to TechData 360
### Link: https://www.youtube.com/@TechData360

## How to Transfer Google Drive Files from One Account to Another (Easy & Fast Guide - 2025)

In this step-by-step tutorial, learn how to easily transfer your Google Drive files from one account to another using Google Colab! Automate the process without downloading or uploading files manually. Perfect for beginners, students, and professionals managing multiple Drive accounts.

In [ ]:
#@title Interactive GDrive Permission Manager
#@markdown ## 1. Authenticate & List Shared Drives
#@markdown First, run this cell. It will connect to your Google account and list all the Shared Drives you have access to.
#@markdown You can copy the name of a Shared Drive from the output to use in the next step.

# --- Installation & Authentication ---
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib -q
from google.colab import auth
from googleapiclient.discovery import build
import google.auth
from googleapiclient.http import BatchHttpRequest
import time
from tqdm.notebook import tqdm
import os

# Authenticate the user
auth.authenticate_user()

# Get default credentials and build the Drive service
creds, _ = google.auth.default()
drive_service = build('drive', 'v3', credentials=creds)

print("✅ Authentication successful.")

# --- List Shared Drives for user convenience ---
try:
    print("\n--- Available Shared Drives ---")
    drives_response = drive_service.drives().list().execute()
    shared_drives = drives_response.get('drives', [])
    if not shared_drives:
        print("No Shared Drives found.")
    else:
        for drive in shared_drives:
            print(f"- Name: '{drive.get('name')}' (ID: {drive.get('id')})")
    print("-----------------------------\n")
except Exception as e:
    print(f"⚠️ Could not list Shared Drives: {e}")


#@markdown ---
#@markdown ## 2. Configure & Run Action
#@markdown Fill out the form below. If you want to act on an item in a Shared Drive, paste its name from the list above. Otherwise, leave the 'shared_drive_name' field blank to use your 'My Drive'.

#@markdown ### **Action**
#@markdown Choose whether to add or remove permissions.
action = "Share" #@param ["Share", "Unshare"]

#@markdown ### **Source Drive (Optional)**
#@markdown Enter the exact name of the Shared Drive. **Leave blank for 'My Drive'.**
shared_drive_name = "" #@param {type:"string"}

#@markdown ### **Source Path**
#@markdown Enter the full path to the file or folder, relative to the selected drive.
#@markdown *Example (folder): `/My Project Folder`*
#@markdown *Example (file): `/My Project Folder/document.docx`*
source_path = "" #@param {type:"string"}

#@markdown ### **Recipient's Email**
#@markdown Enter the email address of the user whose permissions you want to manage.
destination_email = "" #@param {type:"string"}

#@markdown ### **Sharing Scope (for folders only)**
#@markdown If the path is a folder, choose what to affect within it.
sharing_scope = "Files and Folders" #@param ["Files and Folders", "Files Only", "Folders Only"]

#@markdown ### **Sharing Role (for 'Share' action only)**
#@markdown Choose the level of permission to grant the recipient.
sharing_role = "writer" #@param ["writer", "commenter", "reader"]

#@markdown ### **Notifications (for 'Share' action only)**
#@markdown Check this box if you want the recipient to receive an email notification.
send_notification = False #@param {type:"boolean"}


def get_item_id_from_path(drive_id, is_shared_drive, path):
    """
    Translates a path into a Google Drive file/folder ID and its mimeType,
    supporting both My Drive and Shared Drives.
    """
    clean_path = path.strip().strip('/')
    path_components = clean_path.split('/') if clean_path else []

    current_id = drive_id
    item_info = None

    for component in path_components:
        if not component: continue
        query = f"name='{component}' and '{current_id}' in parents and trashed=false"
        try:
            list_request = drive_service.files().list(
                q=query, fields='files(id, name, mimeType)', pageSize=2,
                supportsAllDrives=True, includeItemsFromAllDrives=True)
            if is_shared_drive:
                list_request.driveId = drive_id
                list_request.corpora = 'drive'
            else:
                 list_request.corpora = 'user'
            response = list_request.execute()
            files = response.get('files', [])
            if not files:
                print(f"❌ ERROR: Path component '{component}' not found.")
                return None, None
            if len(files) > 1: print(f"⚠️ Warning: Multiple items named '{component}' found. Using the first one.")
            item_info = files[0]
            current_id = item_info['id']
        except Exception as e:
            print(f"❌ An error occurred while resolving path component '{component}': {e}")
            return None, None

    if not path_components:
        item_info = drive_service.files().get(fileId=drive_id, fields='id, name, mimeType', supportsAllDrives=True).execute()

    return item_info.get('id'), item_info.get('mimeType')

def manage_permissions_by_path():
    """
    Finds the item at the specified path and shares or unshares it (or its contents).
    """
    # --- Step 1: Determine the target drive ---
    target_drive_id = 'root'
    is_shared_drive = False
    drive_display_name = "'My Drive'"

    if shared_drive_name:
        print(f"Searching for Shared Drive: '{shared_drive_name}'...")
        try:
            drive_response = drive_service.drives().list(q=f"name = '{shared_drive_name}'").execute()
            drives = drive_response.get('drives', [])
            if not drives:
                print(f"❌ ERROR: Shared Drive '{shared_drive_name}' not found.")
                return
            target_drive_id = drives[0]['id']
            drive_display_name = f"Shared Drive '{shared_drive_name}'"
            is_shared_drive = True
            print(f"✅ Found {drive_display_name} (ID: {target_drive_id})")
        except Exception as e:
            print(f"❌ An error occurred while searching for the Shared Drive: {e}")
            return

    # --- Step 2: Resolve the path to an item ID ---
    print(f"\nResolving path: '{source_path}' in {drive_display_name}...")
    item_id, mime_type = get_item_id_from_path(target_drive_id, is_shared_drive, source_path)
    if not item_id: return

    # --- Step 3: Get list of items to process based on path and scope ---
    items_to_process = []
    if mime_type == 'application/vnd.google-apps.folder':
        print(f"✅ Path points to a folder. Applying scope: '{sharing_scope}'.")
        base_query = f"'{item_id}' in parents and trashed=false"
        scope_query = ""
        if sharing_scope == "Files Only": scope_query = " and mimeType != 'application/vnd.google-apps.folder'"
        elif sharing_scope == "Folders Only": scope_query = " and mimeType = 'application/vnd.google-apps.folder'"
        list_request = drive_service.files().list(
            q=base_query + scope_query, fields="nextPageToken, files(id, name)", pageSize=1000,
            supportsAllDrives=True, includeItemsFromAllDrives=True)
        if is_shared_drive: list_request.driveId = target_drive_id; list_request.corpora = 'drive'
        else: list_request.corpora = 'user'
        page_token = None
        while True:
            try:
                if page_token: list_request.pageToken = page_token
                response = list_request.execute()
                items_to_process.extend(response.get('files', []))
                page_token = response.get('nextPageToken', None)
                if page_token is None: break
            except Exception as e:
                print(f"❌ An error occurred while listing folder contents: {e}")
                return
    else:
        print("✅ Path points to a single file.")
        item_name_req = drive_service.files().get(fileId=item_id, fields='name', supportsAllDrives=True).execute()
        items_to_process.append({'id': item_id, 'name': item_name_req.get('name', 'Unknown File')})

    if not items_to_process:
        print("📁 No items found to process based on the provided path and scope.")
        return

    # --- Step 4: Execute the chosen action ---
    print(f"\nPreparing to {action.upper()} {len(items_to_process)} item(s) for '{destination_email}'...")

    if action == "Share":
        permission = {'type': 'user', 'role': sharing_role, 'emailAddress': destination_email}
        batch = drive_service.new_batch_http_request()
        for item in items_to_process:
            batch.add(drive_service.permissions().create(
                fileId=item['id'], body=permission,
                sendNotificationEmail=send_notification,
                supportsAllDrives=True))
        print("Executing batch share request...")
        try:
            batch.execute()
        except Exception as e:
            print(f"❌ Batch sharing failed: {e}")

    elif action == "Unshare":
        permissions_to_delete = []
        print("Finding existing permissions to remove...")
        for item in tqdm(items_to_process, desc="Checking permissions"):
            try:
                perms = drive_service.permissions().list(fileId=item['id'], fields='permissions(id, emailAddress)', supportsAllDrives=True).execute()
                for p in perms.get('permissions', []):
                    if p.get('emailAddress', '').lower() == destination_email.lower():
                        permissions_to_delete.append({'fileId': item['id'], 'permissionId': p['id']})
                        break
            except Exception as e:
                print(f"\n⚠️ Could not check permissions for '{item['name']}'. Skipping. Reason: {e}")

        if not permissions_to_delete:
            print(f"No permissions found for '{destination_email}' on the selected items.")
            return

        print(f"\nFound {len(permissions_to_delete)} permissions to remove. Executing batch delete request...")
        batch = drive_service.new_batch_http_request()
        for perm in permissions_to_delete:
            batch.add(drive_service.permissions().delete(fileId=perm['fileId'], permissionId=perm['permissionId'], supportsAllDrives=True))
        try:
            batch.execute()
        except Exception as e:
            print(f"❌ Batch unsharing failed: {e}")

    print(f"\n✅ All {len(items_to_process)} items have been processed.")

# --- Execute the Main Function ---
if destination_email and "@" in destination_email:
    manage_permissions_by_path()
else:
    print("Please provide a valid recipient email address before running.")


In [ ]:
#@title Interactive 'Shared with Me' to My Drive Copier
#@markdown ## 1. Authenticate & Set Up
#@markdown First, run this cell to install necessary libraries, connect to your Google account, and set up the Drive service with a long timeout for large operations.

# --- Installation & Authentication ---
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib -q
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import BatchHttpRequest
import google.auth
from google.auth.transport.requests import Request
import requests
import time
import math
from tqdm.notebook import tqdm

# --- Authentication ---
print("Authenticating user...")
auth.authenticate_user()

# --- Drive Service Setup with Increased Timeout ---
print("Setting up Google Drive service...")
creds, _ = google.auth.default()
# Use a session with a long timeout (600s = 10 minutes) for network operations
session = requests.Session()
session.timeout = 600
transport = Request(session)
creds.refresh(transport)
# Build the service with the credentials containing the timeout config
drive_service = build('drive', 'v3', credentials=creds)
print("✅ Authentication successful. You can now configure the copy process below.")

#@markdown ---
#@markdown ## 2. Configure & Run Copy Process
#@markdown Fill out the form below, then run this cell to start copying.

#@markdown ### **Source Item (Optional)**
#@markdown To copy a **specific item**, enter its exact name here.
#@markdown **Leave this blank to process ALL items** in your 'Shared with me' list.
specific_item_name = "" #@param {type:"string"}

#@markdown ### **Destination Folder**
#@markdown Enter the name of a folder in your 'My Drive' to copy items into.
#@markdown **Leave this blank to copy to the main 'My Drive' root.**
destination_folder_name = "" #@param {type:"string"}

#@markdown ### **Content Scope**
#@markdown If copying all items or a specific folder, choose what to copy.
copy_scope = "Files and Folders" #@param ["Files and Folders", "Files Only", "Folders Only"]

#@markdown ### **Behavior**
#@markdown Check this box to avoid re-copying files/folders that already exist by name in the destination.
skip_existing_items = True #@param {type:"boolean"}

#@markdown ### **Advanced Settings**
#@markdown Adjust these settings for performance tuning on very large or slow transfers.
max_retries_per_chunk = 5 #@param {type:"integer"}
initial_retry_delay_seconds = 15 #@param {type:"integer"}


# --- Helper function to get existing items in a folder ---
def get_existing_items(folder_id):
    """Gets all file and folder names within a given folder for quick lookup."""
    existing_items = {}
    page_token = None
    query = f"'{folder_id}' in parents and trashed = false"
    while True:
        try:
            response = drive_service.files().list(
                q=query,
                fields="nextPageToken, files(id, name, mimeType)",
                pageSize=1000,
                pageToken=page_token
            ).execute()
            for item in response.get('files', []):
                if item['name'] not in existing_items:
                    existing_items[item['name']] = {'id': item['id'], 'mimeType': item['mimeType']}
            page_token = response.get('nextPageToken', None)
            if page_token is None: break
        except Exception as e:
            print(f"  - Warning: Could not list existing items in folder {folder_id}: {e}")
            break
    return existing_items

# --- Function to create batches in chunks with robust retries ---
def execute_batch_in_chunks(items, build_request_func):
    """Executes a list of items in batches with retry logic and exponential backoff."""
    batch_size_limit = 999
    num_items = len(items)
    if num_items == 0: return

    num_batches = math.ceil(num_items / batch_size_limit)
    progress_bar = tqdm(total=num_items, desc="Batch Processing")

    for i in range(num_batches):
        start_index = i * batch_size_limit
        end_index = start_index + batch_size_limit
        chunk = items[start_index:end_index]

        retry_delay = initial_retry_delay_seconds
        for attempt in range(max_retries_per_chunk):
            batch = drive_service.new_batch_http_request()
            for item in chunk:
                build_request_func(batch, item)

            try:
                if len(batch._requests) > 0:
                    batch.execute()
                progress_bar.update(len(chunk))
                break
            except Exception as e:
                print(f"      - Attempt {attempt + 1}/{max_retries_per_chunk} failed for a batch: {e}")
                if attempt + 1 == max_retries_per_chunk:
                    print(f"      - Max retries reached. Skipping this chunk of {len(chunk)} items.")
                    progress_bar.update(len(chunk)) # Mark as processed to not hang the bar
                    break
                print(f"      - Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
                retry_delay *= 2
    progress_bar.close()

# --- Function to recursively copy folder contents ---
def copy_folder_contents(source_folder_id, destination_folder_id, source_folder_name):
    """Recursively and efficiently copies the contents of a source folder."""
    print(f"\n📁 Processing subfolder: {source_folder_name}")

    # List items in source folder
    source_files, source_folders = [], []
    page_token = None
    while True:
        try:
            response = drive_service.files().list(
                q=f"'{source_folder_id}' in parents and trashed = false",
                fields="nextPageToken, files(id, name, mimeType)", pageSize=1000, pageToken=page_token).execute()
            for item in response.get('files', []):
                (source_folders if item['mimeType'] == 'application/vnd.google-apps.folder' else source_files).append(item)
            page_token = response.get('nextPageToken', None)
            if page_token is None: break
        except Exception as e:
            print(f"  - Error listing source contents for '{source_folder_name}': {e}. Aborting this folder.")
            return

    if copy_scope == "Folders Only":
        source_files = [] # Don't copy files if only folders are selected

    existing_dest_items = get_existing_items(destination_folder_id) if skip_existing_items else {}
    files_to_batch = [f for f in source_files if f['name'] not in existing_dest_items]
    folders_to_batch = [f for f in source_folders if f['name'] not in existing_dest_items]
    print(f"  - Found {len(files_to_batch)} new files and {len(folders_to_batch)} new folders to copy.")

    if files_to_batch:
        def build_file_copy_request(batch, file_item):
            batch.add(drive_service.files().copy(fileId=file_item['id'], body={'name': file_item['name'], 'parents': [destination_folder_id]}))
        execute_batch_in_chunks(files_to_batch, build_file_copy_request)

    newly_created_folders = {}
    if folders_to_batch:
        def folder_creation_callback(request_id, response, exception):
            if not exception: newly_created_folders[request_id] = response
        def build_folder_create_request(batch, folder_item):
            metadata = {'name': folder_item['name'], 'mimeType': 'application/vnd.google-apps.folder', 'parents': [destination_folder_id]}
            batch.add(drive_service.files().create(body=metadata, fields='id, name'), request_id=folder_item['id'], callback=folder_creation_callback)
        execute_batch_in_chunks(folders_to_batch, build_folder_create_request)

    for source_folder in source_folders:
        dest_folder_id_for_recursion = None
        if source_folder['id'] in newly_created_folders:
            dest_folder_id_for_recursion = newly_created_folders[source_folder['id']]['id']
        elif skip_existing_items and source_folder['name'] in existing_dest_items:
            dest_folder_id_for_recursion = existing_dest_items[source_folder['name']]['id']
        if dest_folder_id_for_recursion:
            copy_folder_contents(source_folder['id'], dest_folder_id_for_recursion, source_folder['name'])


# --- Main script execution ---
def main():
    print("--- Starting 'Shared with Me' Copy Process ---")
    # Step 1: Determine the destination folder ID
    if destination_folder_name:
        print(f"Searching for destination folder: '{destination_folder_name}'...")
        try:
            response = drive_service.files().list(
                q=f"name='{destination_folder_name}' and mimeType='application/vnd.google-apps.folder' and 'root' in parents and trashed=false",
                fields='files(id, name)').execute()
            if not response['files']:
                print(f"❌ Destination folder '{destination_folder_name}' not found in 'My Drive'. Please create it first.")
                return
            dest_root_id = response['files'][0]['id']
            print(f"✅ Destination set to '{destination_folder_name}' (ID: {dest_root_id})")
        except Exception as e:
            print(f"❌ Error finding destination folder: {e}")
            return
    else:
        dest_root_id = drive_service.files().get(fileId='root', fields='id').execute()['id']
        print("✅ Destination set to the root of 'My Drive'.")

    # Step 2: Get the list of items to process
    items_to_process = []
    if specific_item_name:
        print(f"\nSearching 'Shared with me' for an item named: '{specific_item_name}'...")
        query = f"name = '{specific_item_name}' and sharedWithMe and trashed = false"
        try:
            response = drive_service.files().list(q=query, fields="files(id, name, mimeType)", pageSize=10).execute()
            found_items = response.get('files', [])
            if not found_items:
                print(f"❌ No item named '{specific_item_name}' found in your 'Shared with me' list.")
                return
            if len(found_items) > 1:
                print(f"⚠️ Warning: Found {len(found_items)} items named '{specific_item_name}'. Processing the first one found.")
            items_to_process.append(found_items[0])
            print(f"✅ Found specific item to process.")
        except Exception as e:
            print(f"❌ An error occurred while searching for the specific item: {e}")
            return
    else:
        print("\nListing all items in 'Shared with me'. This may take a moment...")
        page_token = None
        while True:
            try:
                response = drive_service.files().list(q="sharedWithMe and trashed = false",
                    fields="nextPageToken, files(id, name, mimeType)", pageSize=1000, pageToken=page_token).execute()
                items_to_process.extend(response.get('files', []))
                page_token = response.get('nextPageToken', None)
                if page_token is None: break
            except Exception as e:
                print(f"❌ An error occurred while listing all shared items: {e}")
                return

    # Step 3: Filter the list based on scope and existence
    shared_files = [item for item in items_to_process if item['mimeType'] != 'application/vnd.google-apps.folder']
    shared_folders = [item for item in items_to_process if item['mimeType'] == 'application/vnd.google-apps.folder']

    if copy_scope == "Files Only": shared_folders = []
    elif copy_scope == "Folders Only": shared_files = []

    print(f"\nFound {len(shared_files)} files and {len(shared_folders)} folders to process.")
    existing_top_level_items = get_existing_items(dest_root_id) if skip_existing_items else {}
    files_to_copy = [f for f in shared_files if f['name'] not in existing_top_level_items]
    folders_to_copy = [f for f in shared_folders if f['name'] not in existing_top_level_items]
    print(f"Will copy {len(files_to_copy)} new files and {len(folders_to_copy)} new folders to the destination.")

    # Step 4: Process the filtered lists
    if files_to_copy:
        print("\nCopying top-level files...")
        def build_top_file_copy_request(batch, file_item):
            batch.add(drive_service.files().copy(fileId=file_item['id'], body={'name': file_item['name'], 'parents': [dest_root_id]}))
        execute_batch_in_chunks(files_to_copy, build_top_file_copy_request)

    newly_created_top_folders = {}
    if folders_to_copy:
        print("\nCreating top-level folders...")
        def top_folder_callback(request_id, response, exception):
             if not exception: newly_created_top_folders[request_id] = response
        def build_top_folder_create_request(batch, folder_item):
            metadata = {'name': folder_item['name'], 'mimeType': 'application/vnd.google-apps.folder', 'parents': [dest_root_id]}
            batch.add(drive_service.files().create(body=metadata, fields='id, name'), request_id=folder_item['id'], callback=top_folder_callback)
        execute_batch_in_chunks(folders_to_copy, build_top_folder_create_request)

    for folder in folders_to_copy:
        if folder['id'] in newly_created_top_folders:
            dest_folder_id = newly_created_top_folders[folder['id']]['id']
            copy_folder_contents(folder['id'], dest_folder_id, folder['name'])

    print("\n\n✅ Process complete!")


# --- Run the main function ---
if __name__ == "__main__":
    main()
